In [ ]:
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms.functional import to_tensor
from tqdm import tqdm
from sklearn.metrics import accuracy_score


zip_path = 'cats_and_dogs_filtered.zip'
extract_path = './cats_and_dogs_filtered'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(root=os.path.join(extract_path, 'cats_and_dogs_filtered/train'), transform=data_transform)
val_dataset = datasets.ImageFolder(root=os.path.join(extract_path, 'cats_and_dogs_filtered/validation'), transform=data_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

class CatDogClassifier(nn.Module):
    def __init__(self):
        super(CatDogClassifier, self).__init__()
        self.fc1 = nn.Linear(3 * 64 * 64, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

epochs = 10
learning_rate = 0.001
weight_decay = 0.01

model = CatDogClassifier()

optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs} (Training)'):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        l2_reg = torch.tensor(0.0)
        for param in model.parameters():
            l2_reg += torch.norm(param, p=2)
        loss += 0.5 * weight_decay * l2_reg 

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{epochs}, Training Loss: {running_loss / len(train_loader)}')

    # Validation
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{epochs} (Validation)'):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.numpy())
            all_preds.extend(preds.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch {epoch + 1}/{epochs}, Validation Accuracy: {accuracy * 100:.2f}%')


Epoch 1/10 (Training): 100%|████████████████████| 32/32 [00:55<00:00,  1.74s/it]


Epoch 1/10, Training Loss: 0.7980551868677139


Epoch 1/10 (Validation): 100%|██████████████████| 16/16 [00:08<00:00,  1.88it/s]


Epoch 1/10, Validation Accuracy: 50.80%


Epoch 2/10 (Training):  81%|████████████████▎   | 26/32 [00:43<00:10,  1.69s/it]

In [ ]:
epochs = 10
learning_rate = 0.001
l1_lambda = 0.01 

optimizer = optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs} (Training)'):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        l1_reg = torch.tensor(0.0, device=device)
        for param in model.parameters():
            l1_reg += torch.norm(param, p=1) 
        loss += l1_lambda * l1_reg 
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{epochs}, Training Loss: {running_loss / len(train_loader)}')

    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{epochs} (Validation)'):
            # Move the data to the GPU
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch {epoch + 1}/{epochs}, Validation Accuracy: {accuracy * 100:.2f}%')



In [ ]:

class CatDogClassifierWithDropout(nn.Module):
    def __init__(self):
        super(CatDogClassifierWithDropout, self).__init__()
        self.fc1 = nn.Linear(3 * 64 * 64, 512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

model_with_dropout = CatDogClassifierWithDropout().to(device)

epochs = 10
learning_rate = 0.001

optimizer_with_dropout = optim.SGD(model_with_dropout.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model_with_dropout.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs} (Training with Dropout)'):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_with_dropout.zero_grad()
        outputs = model_with_dropout(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_with_dropout.step()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{epochs}, Training Loss with Dropout: {running_loss / len(train_loader)}')

    model_with_dropout.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{epochs} (Validation with Dropout)'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_with_dropout(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch {epoch + 1}/{epochs}, Validation Accuracy with Dropout: {accuracy * 100:.2f}%')
